In [89]:
pip install leafmap


Note: you may need to restart the kernel to use updated packages.


In [90]:
pip install nbformat

Note: you may need to restart the kernel to use updated packages.


In [91]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output


# Utility Function to Read API Responses

def read_response(response):
    try:
        return response.json()
    except requests.exceptions.JSONDecodeError:
        print("Invalid JSON response!")
        print("Response content:", response.text)
        return None


# Get List of Available Provinces

def get_provinces_list():
    response = requests.get("http://127.0.0.1:5000/api/provinces")
    return read_response(response) or []


# Get List of Available Municipalities

def get_municipalities_list():
    response = requests.get("http://127.0.0.1:5000/api/municipalities")
    return read_response(response) or []


# Get List of Available Pollutants

def get_pollutants_list():
    response = requests.get("http://127.0.0.1:5000/api/pollutants")
    return read_response(response) or []


# Call API to Get Data

def get_data(tipo, nome, pollutant):
    url = "http://127.0.0.1:5000/api/DV_7comune" if tipo == "Comune" else "http://127.0.0.1:5000/api/DV_7provincia"
    payload = {
        "var_comune": nome if tipo == "Comune" else None,
        "var_provincia": nome if tipo == "Provincia" else None,
        "var_pollutant": pollutant
    }
    payload = {k: v for k, v in payload.items() if v is not None}
    response = requests.post(url, json=payload)
    data = read_response(response)
    return pd.DataFrame(data) if data else pd.DataFrame()


# Dropdowns e Output UI (creazione degli elementi interattivi)

tipo_dropdown = widgets.Dropdown(
    options=["Comune", "Provincia"],
    description="Type:",
    value="Comune"
)

nome_dropdown = widgets.Dropdown(
    options=[],
    description="Name:"
)

pollutant_dropdown = widgets.Dropdown(
    options=get_pollutants_list(),
    description="pollutant:"
)

output = widgets.Output()

## DV-7 Pollutant comparison over same area
The interface allow the user to select one province or one municipality and visualize the time series of each pollutant, the time series will be obtained calculating for each pollutant the daily average over the selected area.

In [92]:

# Update list of names (comuni or provinces) (Questa funzione si attiva quando cambia la selezione nel dropdown)

def aggiorna_lista_nomi(change):
    if change['new'] == "Comune":
        nome_dropdown.options = get_municipalities_list()
    else:
        nome_dropdown.options = get_provinces_list()

tipo_dropdown.observe(aggiorna_lista_nomi, names='value')

# First loading
aggiorna_lista_nomi({'new': tipo_dropdown.value})


# Show data and graph

def mostra_dati(b):
    with output:
        clear_output() #rimuove eventuali risultati precedenti
        tipo = tipo_dropdown.value 
        nome = nome_dropdown.value
        inquinante = pollutant_dropdown.value #leggo tutti gli elementi dei 3 dropdown

        if not nome:
            print("Please select a valid name.")
            return
        if not inquinante:
            print("Select a pollutant.")
            return #se nome o inquinante non sono selezionati ci dice di selezionare un nome o un inquinante valido 

        df = get_data(tipo, nome, inquinante) #richiamo funzione 
        if df.empty: #se il dataframe è vuoto
            print(f"No data found for {tipo} '{nome}' e inquinante '{inquinante}'.")
            return

        display(df)

        # Plot
        df['data'] = pd.to_datetime(df['data']) #converte data nel formato datatime
        df = df.sort_values('data') #ordina dati per data
        plt.figure(figsize=(10, 4))
        plt.plot(df['data'], df['valore'], marker='.')
        plt.title(f"Time series of {inquinante} - {tipo}: {nome}")
        plt.xlabel("Date")
        plt.ylabel("Average value")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

# -----------------------------
# button and display
# -----------------------------
button = widgets.Button(description="Show data")
button.on_click(mostra_dati)

display(tipo_dropdown, nome_dropdown, pollutant_dropdown, button, output)

Dropdown(description='Type:', options=('Comune', 'Provincia'), value='Comune')

Dropdown(description='Name:', options=('Abbadia Cerreto', 'Abbadia Lariana', 'Abbiategrasso', 'Acquafredda', '…

Dropdown(description='pollutant:', options=('Ammoniaca', 'Arsenico', 'Benzene', 'Benzo(a)pirene', 'Biossido di…

Button(description='Show data', style=ButtonStyle())

Output()

## DV-8 Average concentration – map
The interface allow the user to choose a pollutant and a time window and visualize the average concentration of the selected pollutant over the selected time

In [93]:
# -----------------------------
# Call API to Get DV-8 Data
# -----------------------------
def get_dv8_data(pollutant, start_date, end_date):
    url = "http://127.0.0.1:5000/api/DV_8"
    payload = {
        "pollutant": pollutant,
        "start_date": start_date.isoformat(),
        "end_date": end_date.isoformat()
    } #qui preparo il PAYLOAD ovvero i dati da mandare alla POST
    response = requests.post(url, json=payload) #manda richiesta
    data = read_response(response) #usa funzione per leggere la risposta
    return pd.DataFrame(data) if data else pd.DataFrame()

# -----------------------------
# Widgets for DV-8
# -----------------------------
pollutant_dropdown_dv8 = widgets.Dropdown(
    options=get_pollutants_list(),
    description="Pollutant:"
) #dropdown per selezionare l'inquinante

start_date_picker = widgets.DatePicker(
    description='Start day:',
    disabled=False
) #dropdown per selezionare data

end_date_picker = widgets.DatePicker(
    description='End day:',
    disabled=False
)#dropdown per selezionare data

button_dv8 = widgets.Button(
    description="Show resoults"
)

output_dv8 = widgets.Output()


# Event Handler for DV-8

def on_dv8_button_click(b):
    with output_dv8:
        clear_output() #pulisce output
        pollutant = pollutant_dropdown_dv8.value
        start_date = start_date_picker.value
        end_date = end_date_picker.value #leggo i valori del widjet

        if not pollutant or not start_date or not end_date:
            print("Select pollutant, start date and end date.")
            return

        if start_date > end_date:
            print("The start date must be before the end date.")
            return

        df = get_dv8_data(pollutant, start_date, end_date) #richiamo l'api

        if df.empty:
            print("No data found.")
            return

        display(df)

        

button_dv8.on_click(on_dv8_button_click)


# View interface

display(pollutant_dropdown_dv8, start_date_picker, end_date_picker, button_dv8, output_dv8)


Dropdown(description='Pollutant:', options=('Ammoniaca', 'Arsenico', 'Benzene', 'Benzo(a)pirene', 'Biossido di…

DatePicker(value=None, description='Start day:', step=1)

DatePicker(value=None, description='End day:', step=1)

Button(description='Show resoults', style=ButtonStyle())

Output()

In [94]:
import pandas as pd
import geopandas as gpd
import requests
from shapely import wkt
import folium
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.express as px

# -----------------------------
# Helpers
# -----------------------------

def read_response(response):
    if response.status_code == 200:
        return response.json()
    return None

def get_pollutants_list():
    response = requests.get("http://127.0.0.1:5000/api/pollutants")
    return read_response(response) or []

def get_provinces_list():
    response = requests.get("http://127.0.0.1:5000/api/provinces")
    return read_response(response) or []

def get_dv8_data(pollutant, start_date, end_date):
    url = "http://127.0.0.1:5000/api/DV_8"
    payload = {
        "pollutant": pollutant,
        "start_date": start_date.isoformat(),
        "end_date": end_date.isoformat()
    }
    response = requests.post(url, json=payload)
    data = read_response(response)
    return pd.DataFrame(data) if data else pd.DataFrame()

def get_all_province_shapes():
    url = "http://127.0.0.1:5000/api/province_shape"
    response = requests.get(url)
    data = read_response(response)
    if data:
        df = pd.DataFrame(data)
        df["geometry"] = df["geometry_province"].apply(wkt.loads)
        return gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")
    else:
        return gpd.GeoDataFrame(columns=["geometry"], geometry="geometry", crs="EPSG:4326")

# -----------------------------
# Widgets
# -----------------------------

pollutant_dropdown_dv8 = widgets.Dropdown(
    options=get_pollutants_list(),
    description="Inquinante:"
)

start_date_picker = widgets.DatePicker(
    description='Data inizio:',
    disabled=False
)

end_date_picker = widgets.DatePicker(
    description='Data fine:',
    disabled=False
)

button_dv8 = widgets.Button(
    description="Mostra risultati"
)

output_dv8 = widgets.Output()

# -----------------------------
# Event Handler
# -----------------------------

def on_dv8_button_click(b):
    with output_dv8:
        clear_output()

        pollutant = pollutant_dropdown_dv8.value
        start_date = start_date_picker.value
        end_date = end_date_picker.value

        if not pollutant or not start_date or not end_date:
            print("Seleziona inquinante, data inizio e data fine.")
            return

        if start_date > end_date:
            print("La data di inizio deve precedere quella di fine.")
            return

        df = get_dv8_data(pollutant, start_date, end_date)

        if df.empty:
            print("Nessun dato trovato.")
            return

        display(df)

        fig = px.bar(df, x='province', y='average_value',
                     title=f"Media {pollutant} per provincia",
                     labels={'province': 'Provincia', 'average_value': 'Valore medio'})
        fig.show()

        try:
            provinces_gdf = get_all_province_shapes()
            provinces_list = get_provinces_list()

            if len(provinces_list) != len(provinces_gdf):
                print("Numero province e geometrie non corrisponde.")
                return

            provinces_gdf["prov_upper"] = [p.upper().strip() for p in provinces_list]
            provinces_gdf["prov_name"] = provinces_list

            df["prov_upper"] = df["province"].str.upper().str.strip()
            provinces_gdf["has_data"] = provinces_gdf["prov_upper"].isin(df["prov_upper"])

            max_row = df.loc[df["average_value"].idxmax()]
            prov_max = max_row["province"].upper().strip()
            max_val = max_row["average_value"]

            m = folium.Map(location=[45.64, 9.60], zoom_start=8, tiles="cartodbpositron")

            for _, row in provinces_gdf.iterrows():
                geom = row["geometry"]
                nome_prov = row["prov_name"]
                upper_name = row["prov_upper"]
                has_data = row["has_data"]

                if upper_name == prov_max:
                    color = "red"
                    tooltip = f"<b>{nome_prov}</b><br>Più inquinata<br>Media: {max_val:.2f}"
                elif has_data:
                    avg = df[df["prov_upper"] == upper_name]["average_value"].values[0]
                    color = "lightblue"
                    tooltip = f"{nome_prov}<br>Media: {avg:.2f}"
                else:
                    color = "lightgray"
                    tooltip = nome_prov

                folium.GeoJson(
                    geom.__geo_interface__,
                    style_function=lambda feature, c=color: {
                        "fillColor": c,
                        "color": "black",
                        "weight": 1,
                        "fillOpacity": 0.7
                    },
                    tooltip=folium.Tooltip(tooltip)
                ).add_to(m)

            folium.LayerControl().add_to(m)
            display(m)

        except Exception as e:
            print("Errore nella visualizzazione della mappa:", e)

# -----------------------------
# Collegamento e UI
# -----------------------------

button_dv8.on_click(on_dv8_button_click)
display(pollutant_dropdown_dv8, start_date_picker, end_date_picker, button_dv8, output_dv8)

Dropdown(description='Inquinante:', options=('Ammoniaca', 'Arsenico', 'Benzene', 'Benzo(a)pirene', 'Biossido d…

DatePicker(value=None, description='Data inizio:', step=1)

DatePicker(value=None, description='Data fine:', step=1)

Button(description='Mostra risultati', style=ButtonStyle())

Output()

In [97]:
import pandas as pd
import geopandas as gpd
import requests
from shapely import wkt
import folium
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.express as px

# -----------------------------
# Utility to Read API Response
# -----------------------------
def read_response(response):
    if response.status_code == 200:
        return response.json()
    return None

# -----------------------------
# API Calls
# -----------------------------
def get_pollutants_list():
    response = requests.get("http://127.0.0.1:5000/api/pollutants")
    return read_response(response) or []

def get_dv8_data(pollutant, start_date, end_date):
    url = "http://127.0.0.1:5000/api/DV_8"
    payload = {
        "pollutant": pollutant,
        "start_date": start_date.isoformat(),
        "end_date": end_date.isoformat()
    }
    response = requests.post(url, json=payload)
    data = read_response(response)
    return pd.DataFrame(data) if data else pd.DataFrame()

def get_all_province_shapes_v2():
    url = "http://127.0.0.1:5000/api/province_shape_v2"
    response = requests.get(url)
    data = read_response(response)
    if data:
        df = pd.DataFrame(data)
        df["geometry"] = df["geometry_province_wkt"].apply(wkt.loads)
        return gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")
    return gpd.GeoDataFrame(columns=["nome_provincia", "geometry"], geometry="geometry", crs="EPSG:4326")

# -----------------------------
# Widgets for DV-8
# -----------------------------
pollutant_dropdown_dv8 = widgets.Dropdown(
    options=get_pollutants_list(),
    description="Pollutant:"
)

start_date_picker = widgets.DatePicker(
    description='Start date:',
    disabled=False
)

end_date_picker = widgets.DatePicker(
    description='End date:',
    disabled=False
)

button_dv8 = widgets.Button(description="Show results")
output_dv8 = widgets.Output()

# -----------------------------
# Event Handler
# -----------------------------
def on_dv8_button_click(b):
    with output_dv8:
        clear_output()
        pollutant = pollutant_dropdown_dv8.value
        start_date = start_date_picker.value
        end_date = end_date_picker.value

        if not pollutant or not start_date or not end_date:
            print("Select pollutant, start date and end date.")
            return
        if start_date > end_date:
            print("The start date must be before the end date.")
            return

        df = get_dv8_data(pollutant, start_date, end_date)

        if df.empty:
            print("No data found.")
            return

        display(df)

        # Plotly bar chart
        fig = px.bar(df, x='province', y='average_value',
                     title=f"Average {pollutant} by Province",
                     labels={'province': 'Province', 'average_value': 'Average Value'})
        fig.show()

        # Map
        try:
            provinces_gdf = get_all_province_shapes_v2()
            df["prov_upper"] = df["province"].str.upper().str.strip()
            provinces_gdf["prov_upper"] = provinces_gdf["nome_provincia"].str.upper().str.strip()

            merged_gdf = provinces_gdf.merge(df, on="prov_upper", how="left")

            if merged_gdf["average_value"].dropna().empty:
                print("No data available for map.")
                return

            max_row = merged_gdf.loc[merged_gdf["average_value"].idxmax()]
            prov_max = max_row["prov_upper"]
            max_val = max_row["average_value"]

            m = folium.Map(location=[45.64, 9.60], zoom_start=8, tiles="OpenStreetMap")

            for _, row in merged_gdf.iterrows():
                geom = row["geometry"]
                nome_prov = row["nome_provincia"]
                has_data = pd.notna(row["average_value"])

                if row["prov_upper"] == prov_max:
                    color = "red"
                    tooltip = f"<b>{nome_prov}</b><br>Most polluted<br>Average: {max_val:.2f}"
                elif has_data:
                    color = "lightblue"
                    tooltip = f"{nome_prov}<br>Average: {row['average_value']:.2f}"
                else:
                    color = "lightgray"
                    tooltip = nome_prov

                folium.GeoJson(
                    geom.__geo_interface__,
                    style_function=lambda feature, c=color: {
                        "fillColor": c,
                        "color": "black",
                        "weight": 1,
                        "fillOpacity": 0.7
                    },
                    tooltip=folium.Tooltip(tooltip)
                ).add_to(m)

            display(m)

        except Exception as e:
            print("Error displaying map:", e)

# -----------------------------
# Display UI
# -----------------------------
button_dv8.on_click(on_dv8_button_click)
display(pollutant_dropdown_dv8, start_date_picker, end_date_picker, button_dv8, output_dv8)


Dropdown(description='Pollutant:', options=('Ammoniaca', 'Arsenico', 'Benzene', 'Benzo(a)pirene', 'Biossido di…

DatePicker(value=None, description='Start date:', step=1)

DatePicker(value=None, description='End date:', step=1)

Button(description='Show results', style=ButtonStyle())

Output()